In [1]:
from __future__ import division
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import os
import numbeo_scraper as ns
import get_bea_data as gbd
import population_cleanup as pc
%matplotlib inline

In [2]:
# read in population (1790 - 2010) and rj metrics meetup info (2013-2014) and merge df's
pop_df = pc.get_pop_data('data/1790-2010_MASTER.csv')
rj_df = pc.get_rj_data('data/rj_metrics.txt')
new_df = pd.concat([pop_df, rj_df], axis=1)


In [19]:
# clean and join bureau of economic affairs info

raw_bea = gbd.get_bea_data('http://www.bea.gov/newsreleases/regional/gdp_metro/2015/xls/gdp_metro0915.xls')
bea_df = gbd.clean_me(raw_bea)
bea_df = bea_df[:-2]
next_df = pd.concat([new_df, bea_df[bea_df['bea_2014'] > 20000]], axis=1)
# bea_df.shape
# cities = next_df.index


In [4]:
# incorporate numbeo data:

url_prefix = 'http://www.numbeo.com/cost-of-living/region_rankings.jsp?title='
url_suffix = '&region=021'
year_list = ['2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016']

urls = ns.build_urls(year_list)
#soup_can = [get_pages(url) for url in urls]
for url in urls:
    soup_can = ns.get_pages(url)
table_list = [ns.clean_up(soup) for soup in soup_can]
zipped = list(zip(year_list, table_list))
df_dict = ns.build_data_frames(zipped)

for item in year_list:
    columns= ns.fix_em(['Rank','City','Cost of Living Index','Rent Index','Cost of Living Plus Rent Index',
          'Groceries Index','Restaurant Price Index','Local Purchasing Power Index'])
    first_cols = columns[:2]
    first_cols.extend([column + '_{}'.format(item)for column in columns[2:]]) 
    df_dict[item].columns = first_cols    


In [5]:
def clean_up_df(df):
    df['state'] = df['city'].apply(lambda x: x.split(',')[1].strip().lower().replace(' ', '_'))
    df['city'] = df['city'].apply(lambda x: x.split(',')[0].lower().replace(' ', '_'))
    del df['rank']
    return df

In [6]:
df_2009 = clean_up_df(df_dict['2009'])
df_2010 = clean_up_df(df_dict['2010'])
df_2011 = clean_up_df(df_dict['2011'])
df_2012 = clean_up_df(df_dict['2012'])
df_2013 = clean_up_df(df_dict['2013'])
df_2014 = clean_up_df(df_dict['2014'])
df_2015 = clean_up_df(df_dict['2015'])
df_2016 = clean_up_df(df_dict['2016'])

In [7]:
df_list = [df_2009, df_2010, df_2011, df_2012, df_2013, df_2014, df_2015, df_2016]


In [8]:
def merger(df1, df2):
    df_merge = pd.merge(df1, df2,
              left_on=['city', 'state'],
              right_on=['city', 'state'],
              how='outer')
    return df_merge

merged1 = merger(df_list[0], df_list[1])
merged2 = merger(merged1, df_list[2])
merged3 = merger(merged2, df_list[3])
merged4 = merger(merged2, df_list[4])
merged5 = merger(merged2, df_list[5])
merged6 = merger(merged2, df_list[6])
merged7 = merger(merged2, df_list[7])
merged7.set_index('city', inplace=True)
value_list = ['canada', 'bermuda']
merged7 = merged7[~merged7['state'].isin(value_list)]

In [20]:
next_df.index.name = 'city'
next_df.reset_index(inplace=True)
next_df.head()

,city,ST,1950,1960,1970,1980,1990,2000,2010,STPLFIPS_2010,...,Overall Rank,state,bea_state,bea_2009,bea_2010,bea_2011,bea_2012,bea_2013,bea_2014,bea_what_is_this_crap
0,akron,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,OH,27882.0,28660.0,29544.0,30601.0,31317.0,32509.0,79
1,albany_schenectady_troy,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NY,42547.0,43182.0,43968.0,45788.0,47432.0,49521.0,58
2,albuquerque,NM,96815.0,201189.0,244501.0,332920.0,384736.0,448607.0,545852.0,3502000.0,...,NaN,NaN,NM,38094.0,38964.0,39765.0,40868.0,41393.0,42046.0,64
3,allentown_bethlehem_easton,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,PA-NJ,30041.0,31295.0,31973.0,32867.0,33803.0,35417.0,75
4,anaheim,CA,14556.0,104184.0,166408.0,219494.0,266406.0,328014.0,336265.0,602000.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [22]:
merged7.reset_index(inplace=True)
merged7.head()

,city,cost_of_living_index_2009,rent_index_2009,cost_of_living_plus_rent_index_2009,groceries_index_2009,restaurant_price_index_2009,local_purchasing_power_index_2009,state,cost_of_living_index_2010,rent_index_2010,...,cost_of_living_plus_rent_index_2011,groceries_index_2011,restaurant_price_index_2011,local_purchasing_power_index_2011,cost_of_living_index_2016,rent_index_2016,cost_of_living_plus_rent_index_2016,groceries_index_2016,restaurant_price_index_2016,local_purchasing_power_index_2016
0,san_francisco,100.14,117.65,107.09,117.42,105.24,101.50,ca,90.64,81.55,...,90.84,82.76,72.72,107.10,103.36,121.64,112.15,122.07,90.89,105.83
1,new_york,100.00,100.00,100.00,100.00,100.00,100.00,ny,100.00,100.00,...,100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00
2,tampa,NaN,NaN,NaN,NaN,NaN,NaN,fl,109.64,39.98,...,69.65,61.16,62.24,104.76,72.44,37.19,55.50,75.21,73.74,141.77
3,columbus,NaN,NaN,NaN,NaN,NaN,NaN,oh,104.69,42.63,...,NaN,NaN,NaN,NaN,69.81,34.38,52.78,71.01,64.11,133.01
4,portland,NaN,NaN,NaN,NaN,NaN,NaN,or,102.90,46.57,...,NaN,NaN,NaN,NaN,80.04,48.59,64.92,84.28,71.40,121.98


In [24]:
merged7.sort_values('city', inplace=True)
merged7.head()

,city,cost_of_living_index_2009,rent_index_2009,cost_of_living_plus_rent_index_2009,groceries_index_2009,restaurant_price_index_2009,local_purchasing_power_index_2009,state,cost_of_living_index_2010,rent_index_2010,...,cost_of_living_plus_rent_index_2011,groceries_index_2011,restaurant_price_index_2011,local_purchasing_power_index_2011,cost_of_living_index_2016,rent_index_2016,cost_of_living_plus_rent_index_2016,groceries_index_2016,restaurant_price_index_2016,local_purchasing_power_index_2016
43,akron,NaN,NaN,NaN,NaN,NaN,NaN,oh,NaN,NaN,...,NaN,NaN,NaN,NaN,83.02,20.26,52.86,97.18,62.02,116.32
34,albany,NaN,NaN,NaN,NaN,NaN,NaN,ny,NaN,NaN,...,NaN,NaN,NaN,NaN,90.13,28.97,60.74,106.24,80.47,152.24
29,albuquerque,NaN,NaN,NaN,NaN,NaN,NaN,nm,NaN,NaN,...,57.85,75.64,58.29,116.79,63.81,24.66,45.00,64.55,63.69,153.15
33,allentown,NaN,NaN,NaN,NaN,NaN,NaN,pa,NaN,NaN,...,NaN,NaN,NaN,NaN,92.69,34.27,64.61,100.49,73.23,83.96
41,anaheim,NaN,NaN,NaN,NaN,NaN,NaN,ca,NaN,NaN,...,NaN,NaN,NaN,NaN,83.81,49.63,67.38,89.61,80.49,140.75


In [26]:
merged7.shape


(99, 26)

In [27]:
next_df.shape

(200, 33)

In [40]:
recent_pop_df = pd.merge(next_df, merged7, left_on='city', right_on='city', how='outer')

In [41]:
recent_pop_df.head()

,city,ST,1950,1960,1970,1980,1990,2000,2010,STPLFIPS_2010,...,cost_of_living_plus_rent_index_2011,groceries_index_2011,restaurant_price_index_2011,local_purchasing_power_index_2011,cost_of_living_index_2016,rent_index_2016,cost_of_living_plus_rent_index_2016,groceries_index_2016,restaurant_price_index_2016,local_purchasing_power_index_2016
0,akron,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,83.02,20.26,52.86,97.18,62.02,116.32
1,albany_schenectady_troy,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,albuquerque,NM,96815.0,201189.0,244501.0,332920.0,384736.0,448607.0,545852.0,3502000.0,...,57.85,75.64,58.29,116.79,63.81,24.66,45.00,64.55,63.69,153.15
3,allentown_bethlehem_easton,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,anaheim,CA,14556.0,104184.0,166408.0,219494.0,266406.0,328014.0,336265.0,602000.0,...,NaN,NaN,NaN,NaN,83.81,49.63,67.38,89.61,80.49,140.75


In [42]:
recent_pop_df.set_index('city', inplace=True)

In [43]:
merged1.head()

,city,cost_of_living_index_2009,rent_index_2009,cost_of_living_plus_rent_index_2009,groceries_index_2009,restaurant_price_index_2009,local_purchasing_power_index_2009,state,cost_of_living_index_2010,rent_index_2010,cost_of_living_plus_rent_index_2010,groceries_index_2010,restaurant_price_index_2010,local_purchasing_power_index_2010
0,montreal,101.80,65.44,87.37,87.83,62.93,57.67,canada,85.34,48.39,68.83,90.86,58.18,81.26
1,san_francisco,100.14,117.65,107.09,117.42,105.24,101.50,ca,90.64,81.55,86.58,114.42,90.00,114.16
2,new_york,100.00,100.00,100.00,100.00,100.00,100.00,ny,100.00,100.00,100.00,100.00,100.00,100.00
3,toronto,80.42,59.52,72.13,108.07,57.72,78.54,canada,83.37,52.42,69.53,102.05,62.16,105.33
4,tampa,NaN,NaN,NaN,NaN,NaN,NaN,fl,109.64,39.98,78.51,99.16,66.77,88.87


In [56]:
anaheim_df = pd.read_csv('data/biggestuscities/cities/anaheim_california.csv',
                        header=1)
anaheim_df.index.name =('city')
anaheim_df['city']= 'anaheim'
anaheim_df.set_index('city', inplace=True)
del anaheim_df['0']
anaheim_df

,2015,2014,2013,2012,2011
city,,,,,
anaheim,350742,346997,345012,343248,340559


In [190]:
import glob
a = glob.glob('/Users/IXChris/Desktop/G/capstone/data/biggestuscities/cities/*.csv')
frame_names = [thing.split('/')[-1].split('.')[0].split('_')[0] for thing in a]

In [227]:
df_list =[]
columns =['drop_me', 2015, 2014, 2013, 2012, 2011]
for index, path in enumerate(a):
    temp_df = pd.read_csv(path)
    temp_df.index.name =('city')
    temp_df['city']= frame_names[index]
    temp_df.set_index('city', inplace=True)
    temp_df = temp_df[1:]  
    temp_df.columns = columns
    temp_df.drop('drop_me', axis=1, inplace=True)
    df_list.append(temp_df)

In [228]:
df_list[0]

,2015,2014,2013,2012,2011
city,,,,,
anaheim,350742,346997,345012,343248,340559


In [224]:
first = df_list.pop(0)
first

,drop_me,2015,2014,2013,2012,2011
city,,,,,,
anaheim,1,350742,346997,345012,343248,340559


In [215]:
columns =['drop_me', 2015, 2014, 2013, 2012, 2011]
first.columns =columns

In [225]:
first.drop('drop_me', axis=1)

,2015,2014,2013,2012,2011
city,,,,,
anaheim,350742,346997,345012,343248,340559


In [31]:
cities_next =list(next_df['city'])
cities_merge = list(merged7['city'])
intersect = list(set(cities_next) & set(cities_merge))
len(intersect)

71

In [32]:
union = list(set(cities_next) | set(cities_merge))
len(union)

228

In [35]:
disjoint = list(set(union) - set(intersect))
len(disjoint)
disjoint

['trenton',
 'scranton__wilkes_barre__hazleton',
 u'savannah',
 'spokane_spokane_valley',
 'des_moines_west_des_moines',
 'birmingham',
 'orlando_kissimmee_sanford',
 'newark',
 'allentown_bethlehem_easton',
 'new_york_city',
 'chula_vista',
 'san_jose_sunnyvale_santa_clara',
 'omaha_council_bluffs',
 'henderson',
 'hartford_west_hartford_east_hartford',
 u"st._john's",
 'dallas_fort_worth_arlington',
 'san_bernardino',
 'aurora',
 'fremont',
 'austin_round_rock',
 u'harrisburg',
 'los_angeles_long_beach_anaheim',
 'greenville_anderson_mauldin',
 'norfolk',
 'oxnard_thousand_oaks_ventura',
 'chesapeake',
 u'west_palm_beach',
 u'santa_barbara',
 'milwaukee_waukesha_west_allis',
 'youngstown_warren_boardman',
 'urban_honolulu',
 'glendale',
 u'fairfax',
 'santa_maria_santa_barbara',
 'baltimore_columbia_towson',
 'corpus_christi',
 'hialeah',
 u'stamford',
 u'des_moines',
 'lancaster',
 'denver_aurora_lakewood',
 'jackson',
 u'fort_lauderdale',
 'cape_coral_fort_myers',
 'santa_ana',
 'r

In [230]:
#subset by dense rows
meetup_df = new_df[new_df['Pop'].notnull()]
cities = list(meetup_df.index)

# fix column names
cols = meetup_df.columns
cols = [item.lower().replace(' ', '_') for item in cols]

In [263]:
url = 'https://www.biggestuscities.com/demographics/us/education-college-graduates-by-top-100-city'
cols = ['rank', 'city','state_fb','pct_foreign_born']
df2 = get_grad_data(url, cols, 6)

'/Users/IXChris/Desktop/G/capstone/data/biggestuscities/people_foreign_born_by_top_100_city.csv'